In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [3]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [17]:
# The number of steps within one time segment
TIME_PERIODS = 100
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 100
N_FEATURES = 12

In [18]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'Class_label']


In [19]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [20]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [21]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [22]:
 pd.options.display.float_format = "{:,.5f}".format

In [23]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [38]:
verbose, epochs, batch_size = 0, 100, 16

In [39]:
# def step_decay(epoch):
#    initial_lrate = 0.1
#    drop = 0.1
#    epochs_drop = 16.0
#    lrate = initial_lrate * tf.math.pow(drop,  
#            tf.math.floor((1+epoch)/epochs_drop))
#    return lrate

In [40]:
# lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [41]:
import os
path='/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/sub_wise_process_TT_100ms/'

In [ ]:
for i in range(1,2):
    file_path_train=path+'S'+str(i)+'_'+'tr'+'.'+'csv'
    print(file_path_train)
    df_Train=read_data_Train(file_path_train)
    show_basic_dataframe_info(df_Train)
    df_Train.head(5)
    file_path_test=path+'S'+str(i)+'_'+'tt'+'.'+'csv'
    print(file_path_test)
    df_Test=read_data_Test(file_path_test)
    show_basic_dataframe_info(df_Test)
    df_Test.head(5)
    LABEL = 'ActivityEncoded'
    # Transform the labels from String to Integer via LabelEncoder
    le = preprocessing.LabelEncoder()
    # Add a new column to the existing DataFrame with the encoded values
    df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
    # df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
    df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())
    x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_train shape: ', x_train.shape)
    # print(x_train)
    print(x_train.shape[0], 'training samples')
    print('y_train shape: ', y_train.shape)
    # Set input & output dimensions
    num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
    print('num_time_periods',num_time_periods)
    print('num_sensors',num_sensors)
    num_classes = le.classes_.size
    print('class_list',list(le.classes_))
    # input_shape = (num_time_periods,num_sensors)
    # print(input_shape)
    input_shape = (num_time_periods,num_sensors)
    #x_train = x_train.reshape(x_train.shape[0], input_shape)
    print('x_train shape:', x_train[0].shape)
    print('input_shape:', input_shape)
    x_train = x_train.astype('float32')
    # x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
    # y_train = y_train.astype('float32')
    print('y_train shape',y_train.shape)
    y_train_hot = np_utils.to_categorical(y_train, num_classes)
#     print(y_train_hot)
    # y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
    print('New y_train shape: ', y_train_hot.shape)
    x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
    print('x_test shape: ', x_test.shape)
    # print(x_train)
    print(x_test.shape[0], 'testing samples')
    print('y_test shape: ', y_test.shape)
    # Set input_shape / reshape for Keras
    #x_test = x_test.reshape(x_test.shape[0], input_shape)
    x_test = x_test.astype('float32')
    y_test = y_test.astype('float32')
    y_test_hot = np_utils.to_categorical(y_test, num_classes)
    # n_steps, n_length = 20, 25
    # n_steps, n_length= 10, 50
    # n_steps, n_length= 16, 32
    n_steps, n_length = 100, 12
    n_depth=1
    x_train = x_train.reshape(x_train.shape[0], n_steps, n_length,n_depth)
    print('x_train shape: ', x_train.shape)
    # x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
    # print('x_valid shape: ', x_valid.shape)
    x_test = x_test.reshape(x_test.shape[0], n_steps, n_length,n_depth)
    print('x_test shape: ', x_test.shape)
    n_outputs = y_train_hot.shape[1]
    print('n_outputs',n_outputs)
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size=(9,1), padding='same',kernel_regularizer=l2(1e-04), kernel_initializer="he_normal",strides=(3,1), \
                           input_shape=(n_steps,n_length,n_depth)))
    model.add(MaxPooling2D(pool_size=(8,1),strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Conv2D(filters=64, kernel_size=(5,1),padding="same",kernel_regularizer=l2(1e-04),kernel_initializer="he_normal",strides=(2,1)))
    model.add(Activation('tanh'))
    model.add(Dropout(0.2093))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512, activation='tanh'))
    model.add(Dense(n_outputs, activation='softmax'))
    adam=optimizers.Adam(lr=2.17e-3, beta_1=0.9, beta_2=0.999,epsilon=1e-08, amsgrad=False)
#     sgd=optimizers.SGD(learning_rate=2.17e-3, momentum=0.9, nesterov=False, name='SGD')
    checkpoint_filepath = '/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/checkpoint.hdf5'
    # model.load_weights(checkpoint_filepath) 
    checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
    early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
    print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)
    csv_logger = CSVLogger('/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/CNN100X12/LSTM_nina_20X100.csv', append=True, separator=';')
    history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,early],validation_data=(x_test, y_test_hot), verbose=1)
    best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
    print('epoch_number',best_index+1)
    print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])
    model.load_weights(checkpoint_filepath) 
    _, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
    print('test_accuracy',testaccuracy)
    test_acc.append(testaccuracy)
    print(test_acc)

/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/sub_wise_process_TT_100ms/S1_tr.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 1960000

/media/naveen/nav/mat_codes/nina_DB2_codes/olsson_DB2/sub_wise_process_TT_100ms/S1_tt.csv
Number of columns in the dataframe: 13
Number of rows in the dataframe: 980000

x_train shape:  (19599, 100, 12)
19599 training samples
y_train shape:  (19599,)
num_time_periods 100
num_sensors 12
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0]
x_train shape: (100, 12)
input_shape: (100, 12)
New y_train shape:  (19599, 49)
x_test shape:  (9799, 100, 12)
9799 testing samples
y_test shape:  (9799,)
x_train shape:  (19599, 100, 12, 1)
x_test shape:  (9799, 100, 12, 1)
n_outputs 49
Model: 

Epoch 20/100
1221/1225 [============================>.] - ETA: 0s - loss: 3.8977 - accuracy: 0.0195
Epoch 00020: val_accuracy did not improve from 0.02041
1225/1225 [==============================] - 11s 9ms/step - loss: 3.8978 - accuracy: 0.0194 - val_loss: 3.8960 - val_accuracy: 0.0204
Epoch 21/100
1219/1225 [============================>.] - ETA: 0s - loss: 3.8977 - accuracy: 0.0202
Epoch 00021: val_accuracy did not improve from 0.02041
1225/1225 [==============================] - 10s 8ms/step - loss: 3.8977 - accuracy: 0.0202 - val_loss: 3.8966 - val_accuracy: 0.0204
Epoch 22/100
1225/1225 [==============================] - ETA: 0s - loss: 3.8975 - accuracy: 0.0195
Epoch 00022: val_accuracy did not improve from 0.02041
1225/1225 [==============================] - 10s 8ms/step - loss: 3.8975 - accuracy: 0.0195 - val_loss: 3.8961 - val_accuracy: 0.0204
Epoch 23/100
1219/1225 [============================>.] - ETA: 0s - loss: 3.8977 - accuracy: 0.0189
Epoch 00023: val_accuracy did not

In [ ]:
statistics.mean(test_acc)